# **Ford Vehicle Specifications AI-Chatbot**
A custom AI assistant that answers user queries on technical specifications for Ford’s 2024 car and SUV models. The chatbot uses **Retrieval-Augmented Generation (RAG)**, combining data retrieval from [Ford Motor Company](https://www.ford.com/) and AI-based language generation (`gpt-3.5-turbo-instruct`) to provide precise, contextually relevant answers about Ford vehicle specifications. \

### - Dataset:
I scraped product information for 2024 Car-SUVs from Ford Motor Company's website. Since the base-model gpt-3.5-turbo-instruct is not trained on 2024 data, this makes it a prime example of how RAG technologies can be used to overcome limitations in training data.\
*Copyright (c) 2024 - Amirhossein Nazeri*  \


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
custom_path = "/path/to/your/files"
sys.path.append(custom_path)

In [ ]:
if __name__ == "__main__":
    !pip install pandas
    !pip install BeautifulSoup4
    !pip install sentence_transformers
    !pip install faiss-cpu
    !pip install openai==0.28
    # !pip install PyMuPDF
    # !pip install pdfplumber
    # !pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
import data_scraper
import RAG
import openai_chat
from data_scraper import flattener, load_urls_from_file
from RAG import generate_embeddings, create_faiss_index, query_table_data
from openai_chat import chatbot_response, create_prompt
import os

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# URLs to scrape data from
# urls = data_scraper.load_urls_from_file(custom_path+"data/urls.csv")
urls = ["https://www.ford.com/suvs-crossovers/edge/models/edge-se/",
        "https://www.ford.com/suvs/explorer/2024/models/explorer-st/",
        "https://www.ford.com/suvs/mach-e/models/mach-e-select/",
        "https://www.ford.com/suvs/expedition/models/expedition-xl-stx/",
        "https://www.ford.com/cars/mustang/2024/models/gt-fastback/",
        "https://www.ford.com/suvs-crossovers/escape/2024/models/escape-st-line/",
        "https://www.ford.com/suvs/bronco-sport/models/bronco-sport-big-bend/",
        "https://www.ford.com/suvs/bronco/models/bronco-badlands/"]

In [ ]:
class AI_chatbot():

    def __init__(self, urls, query):
        self.urls = urls
        self.query = query

    def get_web_data(self):
        # Scrape and parse HTML tables
        return flattener(self.urls)

    def get_rag_indices(self):
        # Generate embeddings
        embeddings = generate_embeddings(self.get_web_data())
        # create FAISS index
        return create_faiss_index(embeddings)

    def get_relevant_data(self):
        # Get retrieveal relevant data
        return query_table_data(self.query, self.get_rag_indices(), self.get_web_data())

    def generate_response(self, no_rag = True):
        # Generate response using OpenAI API

        prompt = create_prompt(self.get_relevant_data(), self.query)
        if no_rag:
            prompt = create_prompt("No extra information. asnwer based on your own knowledge", self.query)
        return chatbot_response(prompt)

## **Performance Demonstration**

#### In the section I compare the Q/A results of my AI-chatbot (`gpt-3.5-turbo-instruct + advanced RAG`) versus a base OpenAI chat completion model (`gpt-3.5-turbo-instruct`).

- In all three questions, the custom AI-chatbot outperforms the base-chatbot. In all three senarios base-chatbot tries to get the most probable answer, however it fails as it is not trained on 2024 data.

### **Q1) Explain the engine of Ford Expedition 2024**
*  ### **Custom AI chatbot response:**

In [ ]:
query = "Explain the engine of Ford Expedition 2024."
AI_chatbot_run = AI_chatbot(urls, query)
AI_chatbot_run.generate_response()

'The engine of the 2024 Ford Expedition XL STX SUV is a 3.5L EcoBoost® V6 with gasoline direct injection. It has a die-cast aluminum engine block and a twin independent variable camshaft timing (Ti-VCT) valvetrain. The engine is capable of producing 440 horsepower and 510 lb.-ft of torque when tuned for H.O. performance. The exhaust is single, stainless steel for the XL-STX and XLT models, but'

In [ ]:
AI_chatbot_run = AI_chatbot(urls, query)
AI_chatbot_run.generate_response()

'The 2024 Ford Expedition is expected to feature a powerful engine that will continue to deliver on its reputation for strength and reliability. It is rumored to be equipped with a 3.5-liter twin-turbocharged V6 engine, which is expected to produce around 400 horsepower and 480 lb-ft of torque. This engine will likely be paired with a 10-speed automatic transmission for smooth and efficient performance. Additionally, the 2024 Expedition is expected to have advanced engine technologies such'

### **Q2) What are the exterior dimensions of Ford bronco 2024?**
*  ### **Custom AI chatbot response:**

In [ ]:
AI_chatbot_run.query = "What are the exterior dimensions of Ford bronco 2024?"
print(AI_chatbot_run.generate_response())

'The exterior dimensions of the 2024 Ford Bronco include a length of 172.7 inches, a height of 70.2/71.4 inches (without options), a width of 76.3 inches with folded mirrors, and a width of 74.3 inches without mirrors. The drive system is 4 X 4.'

*  ### **Base AI chatbot response:**

In [ ]:
AI_chatbot_run.query = "What are the exterior dimensions of Ford bronco 2024?"
AI_chatbot_run.generate_response()

'The exterior dimensions of the Ford Bronco 2024 are approximately 190 inches in length, 75 inches in width, and 73 inches in height.'

### **Q3) Compare the hoursepower and torque of Mustangs 2024.**
*  ### **Custom AI chatbot response:**

In [ ]:
AI_chatbot_run.query = "Compare the hoursepower and torque of Mustangs 2024."
print(AI_chatbot_run.generate_response())

Based on the information provided, the 2024 Ford Mustang GT Fastback has a torque of 350 lb.-ft. @ 3,000rpm (93-octane fuel) and a horsepower of 315 @ 5,500rpm (93-octane fuel) in the base model. In the Premium GT model, the torque increases to 415 lb.-ft. @ 4,900rpm (93-octane fuel) while the horsepower bumps up to 480


*  ### **Base AI chatbot response:**

In [ ]:
AI_chatbot_run.query = "Compare the hoursepower and torque of Mustangs 2024"
AI_chatbot_run.generate_response()

'vs Mach 1\n\nFor the 2024 Mustang, the base engine is the 2.3L EcoBoost 4 cylinder engine, which produces 310 horsepower and 350 lb-ft of torque.\n\nFor the Mach 1, the base engine is the 5.0L Coyote V8 engine, which produces 480 horsepower and 420 lb-ft of torque.\n\nTherefore, the Mach 1 has significantly higher horsepower and torque than the 2024 Mustang.'

```
Copyright (c) 2024 as [AI Chatbot: Ford Vehicle Specifications ] by Amirhossein Nazeri.
This code is part of the repository [amirhnazerii/chatbot_LLM] and is provided for educational or  instructional purposes only.
Unauthorized use, modification, or distribution of this code is prohibited without permission, except as allowed by the license provided.
- Created by @Amirhossein Nazeri
```